# Abstract, Introduction, Related works / Background 를 통해 keyword 추출

In [4]:
import pandas as pd
import ray
import openai
import os
import json

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

### load paper parquet data

In [7]:
# 70000개의 논문에 대해서 저장한 것
saved_df = pd.read_parquet('../data/data_files/filtered_data/processed_papers.parquet')

# related_work 혹은 background 둘 다 없는 경우, introduction 없는 경우 모두 제거 후 사용 
nothing_df = saved_df[(saved_df['related_work'].str.startswith("CODE")) & (saved_df['background'].str.startswith("CODE"))]
no_intro = saved_df[saved_df['introduction'].str.startswith("CODE")]

nothig_df_idx = list(nothing_df.index)
no_intro_idx = list(no_intro.index)

union_index = list(set(nothig_df_idx).union(set(no_intro_idx)))
paper_df = saved_df.drop(index=union_index)
paper_df = paper_df.sort_values(by=['published']).reset_index(drop=True)

In [8]:
# 논문 데이터
paper_docs = []
for idx in paper_df.index:
    row_contents = []
    tmp_rows = paper_df.iloc[idx]
    
    row_abstract = tmp_rows['abstract']
    row_introduction = tmp_rows['introduction']
    row_related_work = tmp_rows['related_work']
    row_background = tmp_rows['background']
    
    row_contents.append(f"[ABSTRACT]: {row_abstract}")
    
    if not row_introduction.startswith("CODE"):
        row_contents.append(f"[INTRODUCTION]: {row_introduction}")
    
    if not row_related_work.startswith("CODE"):
        row_contents.append(f"[RELATED_WORKS]: {row_related_work}")
        
    if not row_background.startswith("CODE"):
        row_contents.append(f"[BACKGROUND]: {row_background}")
    
    row_contents_str = ''.join(row_contents)
    paper_docs.append(row_contents_str)

### GPT 사용하여 키워드 추출

In [5]:
# LLM에 원문 입력
def extract_keyword_using_gpt(text):
    prompt = f"""
    문서의 구조는 다음과 같습니다:
    - [ABSTRACT]: 논문의 초록으로 현재 연구의 주제와 핵심 아이디어를 요약합니다.
    - [INTRODUCTION]: 연구의 배경과 목표를 소개하며, 현재 연구의 주요 내용과 동기를 설명합니다.
    - [RELATED_WORKS]: 이전 연구를 요약하고, 현재 연구와의 차별성을 논의합니다.
    - [BACKGROUND]: 연구의 기초 개념과 관련 정보를 제공합니다.

    현재 문서의 내용을 기반으로 아래 작업을 수행하세요:
    1. ABSTRACT와 INTRODUCTION를 참고하여 현재 연구를 가장 잘 대표하는 키워드를 최대 2개 추출하세요.
    2. RELATED WORKS와 BACKGROUND를 참고하여 현재 연구 이전의 선행 연구 키워드를 추출하세요.
    - 이전의 선행 연구 키워드는 1번에서의 대표 키워드와 중복될 수 없다. 
    - 모든 키워드는 특정 언어에 국한되지 않도록 일반화 과정을 추가한다. 
        예시: 'Arabic-Hebrew parallel corpus' 는 'parallel corpus' 와 같이 치환
    - 단 데이터셋(Dataset)이 키워드로 들어가는 경우에는 제거한다.
    3. 최종적으로 1번과 2번 중 연관도가 높은 키워드 5개 추출하여 dictionary 형태로 출력합니다,
    모든 키워드는 영어로 되어있으며, 소문자로 답한다.
    결과를 **JSON 형식**으로만 반환하세요. 텍스트로 된 설명이나 코드 블록은 포함하지 마세요. 예시는 다음과 같습니다:
    {{
        "main_keywords": ["keyword1", "keyword2"],
        "prev_keywords": ["keyword3", "keyword4", "keyword5", "keyword6", "keyword7"]
    }}
    문서: {text}
    """

    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": "학술 논문에 대해서 잘 알고있으며, 논문 내 키워드 관계 분석 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        top_p=1
    )

    return(response.choices[0].message.content.strip())

In [17]:
# sample
for idx in range(len(paper_docs))[222:225]:
    tmp_text = paper_docs[idx]
    tmp_json_result = extract_keyword_using_gpt(tmp_text)
    tmp_result = json.loads(tmp_json_result)
    paper_keywords = tmp_result.get("main_keywords")
    prev_keywords = tmp_result.get("prev_keywords")
    
    paper_df.loc[idx, 'main_keywords'] = str(paper_keywords)
    paper_df.loc[idx, 'prev_keywords'] = str(prev_keywords)

['word embeddings', 'sense embeddings']
['semantic networks', 'lexical resources', 'disambiguation', 'neural architectures', 'vector space']
['entity identification', 'neural architecture']
['named-entity recognition', 'sequence labeling', 'structured prediction', 'multitasking', 'bidirectional lstm']
['neural execution', 'natural language querying']
['symbolic execution', 'reinforcement learning', 'knowledge base', 'neural networks', 'program synthesis']


#### 전체 데이터에 대해서 키워드 추출 (병렬처리를 위해 ray 사용)

In [ ]:
ray.init(num_cpus=5, ignore_reinit_error=True)  # 최대 4개의 CPU 사용

@ray.remote(max_retries=3)
def process_document(idx, text):
    try:
        tmp_json_result = extract_keyword_using_gpt(text)
        tmp_result = json.loads(tmp_json_result)
        paper_keywords = tmp_result.get("main_keywords", [])
        prev_keywords = tmp_result.get("prev_keywords", [])

    except Exception as e:
        print(f"Error at index {idx}: {e}")  # 디버깅 정보 출력
        paper_keywords, prev_keywords = [], []
    return idx, paper_keywords, prev_keywords

# 데이터를 준비
data = [(idx, paper_docs[idx]) for idx in range(len(paper_docs))]

# CPU 수에 맞는 배치 크기 설정
# 너무 크게 잡으면 결과값에 None 나옴
num_cpus = 5
batch_size = max(1, 300 // num_cpus)  # 배치 크기를 CPU 수에 따라 조정

results = []
saved_batches = 0

for start in range(0, len(data), batch_size):
    end = min(start + batch_size, len(data))
    batch = data[start:end]  # batch 데이터를 슬라이스
    futures = [process_document.remote(idx, text) for idx, text in batch]
    batch_results = ray.get(futures)
    
    # 배치 결과 저장
    batch_df = pd.DataFrame(batch_results, columns=["idx", "main_keywords", "prev_keywords"])
    batch_filename = f"../data/data_files/keyword_parquet_backup/results_{start}_{end}.parquet"  # start와 end를 파일명에 포함
    batch_df.to_parquet(batch_filename, index=False)  # 파일로 저장

    print(f"Batch {start}-{end} saved to {batch_filename}.")
    results.extend(batch_results)  # 전체 결과에 추가
    
     # 결과가 1000개 단위마다
    if len(results) % 1000 == 0:
        batch_df = pd.DataFrame(results, columns=["idx", "main_keywords", "prev_keywords"])
        batch_filename = f"../data/data_files/keyword_parquet_per_batch/results_cumulative_batch_{saved_batches}.parquet"
        batch_df.to_parquet(batch_filename, index=False)
        print(f"Cumulative results saved to {batch_filename}.")
        
        results = []  # 저장 후 누적 리스트 초기화
        saved_batches += 1  # 저장된 배치 수 증가

ray.shutdown()

# 디버깅 결과 출력
for idx, paper_keywords, prev_keywords in results:
    if not paper_keywords and not prev_keywords:
        print(f"No keywords found for document {idx}")


In [ ]:
folder_path = '../data/data_files/keyword_parquet_per_batch/'
keyword_df = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith(".parquet") and "results_" in file_name:
        file_path = os.path.join(folder_path, file_name)
        tmp_df = pd.read_parquet(file_path)
        keyword_df = pd.concat([keyword_df, tmp_df], ignore_index=True)

keyword_df = keyword_df.sort_values(by=['idx']).reset_index(drop=True)
keyword_df.to_parquet('../data/data_files/keyword_data/keyword_df.parquet')

In [ ]:
total_df = pd.merge(paper_df, keyword_df, left_index=True, right_index=True)
total_df = total_df.drop(columns=['idx'])
total_df.to_parquet('../data/data_files/keyword_data/total_df_with_keyword.parquet')